### model_name = 'VPrashant/cypher-gen' Use for cypher query generaion from text

In [1]:
import torch
from datasets import load_dataset

from transformers import Trainer, TrainingArguments
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
# Load model and tokenizer
model_name = "./results/"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [4]:
# Prepare data
data = load_dataset('csv', data_files={'train': 'train.csv'})
data = data['train'].train_test_split(test_size=0.2, seed=42)

In [5]:
def preprocess(batch):
    # Handle all examples in the batch
    input_ids = []
    attention_mask = []
    labels = []
    
    for prompt, query in zip(batch['prompt'], batch['query']):
        # Tokenize input and output
        inputs = tokenizer("translate to Cypher: " + prompt, truncation=True, padding='max_length', max_length=128)
        outputs = tokenizer(query, truncation=True, padding='max_length', max_length=128)
        
        # Collect tokenized inputs and labels
        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])
        labels.append(outputs['input_ids'])
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [6]:
tokenized_data = data.map(preprocess, batched=True)
tokenized_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

In [7]:
# Fine-tune the model
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    weight_decay=0.01,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer
)

trainer.train()


/home/rmaurya/merai/cypher/.cypher/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_18358/1004274551.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/1950 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 0.00018672127043828368, 'eval_runtime': 0.5892, 'eval_samples_per_second': 220.644, 'eval_steps_per_second': 28.854, 'epoch': 1.0}


...

  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 1.7332826246274635e-05, 'eval_runtime': 0.6075, 'eval_samples_per_second': 214.007, 'eval_steps_per_second': 27.985, 'epoch': 29.0}


  0%|          | 0/17 [00:00<?, ?it/s]

{'eval_loss': 1.6929872799664736e-05, 'eval_runtime': 0.5633, 'eval_samples_per_second': 230.772, 'eval_steps_per_second': 30.178, 'epoch': 30.0}
{'train_runtime': 281.1879, 'train_samples_per_second': 55.372, 'train_steps_per_second': 6.935, 'train_loss': 0.0008108815856468983, 'epoch': 30.0}


TrainOutput(global_step=1950, training_loss=0.0008108815856468983, metrics={'train_runtime': 281.1879, 'train_samples_per_second': 55.372, 'train_steps_per_second': 6.935, 'total_flos': 526817962229760.0, 'train_loss': 0.0008108815856468983, 'epoch': 30.0})

In [8]:
model.save_pretrained("./results/")
tokenizer.save_pretrained("./results/")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/spiece.model',
 './results/added_tokens.json')

In [13]:
def test(test_input):
    # Load fine-tuned model and tokenizer
    model_name = './results/'  # Path to your fine-tuned model
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Set the model to evaluation mode
    model.eval()

    # Check if GPU is available and move model to GPU if possible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize the input
    test_encoding = tokenizer(
        test_input,
        return_tensors="pt",
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    test_encoding = {key: val.to(device) for key, val in test_encoding.items()}  # Move input to device

    # Generate Cypher query
    with torch.no_grad():  # Disable gradient calculations for inference
        output = model.generate(
            input_ids=test_encoding['input_ids'],
            attention_mask=test_encoding['attention_mask'],
            max_length=128
        )
    
    # Decode the generated query
    generated_query = tokenizer.decode(output[0], skip_special_tokens=True)

    # Print the generated Cypher query
    print("Input:", test_input)
    print("Generated Cypher Query:", generated_query)

# Call the test function
test_input = "give me a list of 10 genes"

test(test_input)


Input: give me a list of 10 genes
Generated Cypher Query: MATCH (g:Gene) RETURN g LIMIT 10
